In [1]:
from qiskit import Aer, QuantumCircuit, ClassicalRegister, QuantumRegister, IBMQ, execute
import csv
import numpy as np
import matplotlib.pyplot
from matplotlib.pyplot import *
from qiskit.tools.visualization import plot_histogram
from matplotlib.lines import Line2D


In [2]:
class CSVWriter:

    def __init__(self, QuantumCircuit, backends, input_bits, error_op, shots, filename):
        self.qc = QuantumCircuit
        self.error_op = error_op
        self.input_bits = input_bits
        self.jobs = []
        self.filename = filename
        self.__init_jobs(backends, shots)

    class __job:

        def __init__(self, SpecificBackend, qc, shots):
            self.qc = qc
            self.shots = shots
            provider = IBMQ.get_provider()
            self.backendname = SpecificBackend
            self.backend = provider.get_backend(SpecificBackend)
            self.result, self.id = self.__run()

        def __run(self):
            job = execute(self.qc, backend = self.backend, shots = self.shots)
            return job.result(), job.job_id()

    def __id(self, job):
        return job.id

    def __machine(self, job):
        return job.backendname

    def __shots(self, job):
        return job.shots

    def __KeyVals(self, job):
        dic = job.result.data()['counts']
        keys, vals = zip(*dic.items())
        return keys, vals

    def __init_jobs(self, backends, shots):
        print('Initializing Jobs...')
        for backend in backends:
            print('Running qc on '+backend+' for '+str(shots)+' shots...')
            self.jobs.append(self.__job(backend, self.qc, shots))
            print('done.')

    def __generate_labels(self, job):
        keys, vals = self.__KeyVals(job)
        labels = [['Machine'], ['input bits'], ['error operator'], ['no. of shots'], ['job id']]
        for key in keys:
            labels.append([key])
        return labels

    def __generate_data(self, job, Ldat):
        keys, vals = self.__KeyVals(job)
        Ldat[0].append(self.__machine(job))
        Ldat[1].append(self.input_bits)
        Ldat[2].append(self.error_op)
        Ldat[3].append(self.__shots(job))
        Ldat[4].append(self.__id(job))
        for i in range(len(keys)):
            Ldat[5+i].append(vals[i])
        
    def generate_csv(self):
        filename = self.filename
        labels = self.__generate_labels(self.jobs[0])
        for job in self.jobs:
            self.__generate_data(job, labels)
        with open(filename, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(labels)

In [2]:
#Loading IBMQ Account
IBMQ.save_account('(INSERT USER IBMQ TOKEN HERE)', overwrite=True)
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [10]:
# Then we construct our quantum circuit however we desire.
##Build the Circuit and Visualize   
##STANDARD decomposition with W=H

t = np.arcsin(np.sqrt(1/3)) #t=theta/2
a = np.pi/4
b = np.pi-t

#Init Circuit
q = QuantumRegister(3)
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)

## encoding ##
qc.x(q[1])
qc.cu3(np.pi+t+t,0,np.pi,q[1],q[2])
qc.x(q[1])
qc.x(q[2])
qc.cry(np.pi/2,q[2],q[1])
qc.x(q[2])
qc.z(q[0])
qc.cx(q[0],q[1])
qc.cx(q[2],q[0])
qc.x(q[1])
qc.cx(q[1],q[2])
qc.x(q[1])
 
qc.barrier()    
##error operator##
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])

qc.barrier()
## decoding ##
qc.x(q[1])
qc.cx(q[1],q[2])
qc.x(q[1])
qc.cx(q[2],q[0])
qc.cx(q[0],q[1])
qc.z(q[0])
qc.x(q[2])
qc.cry(-np.pi/2,q[2],q[1])
qc.x(q[2])
qc.x(q[1])
qc.cu3(np.pi+t+t,0,np.pi,q[1],q[2])
qc.x(q[1])

#measurement
qc.barrier()
qc.measure(q[1],c[0])


CircFig=qc.draw(output="latex",filename='STDCircFig.png')

In [5]:
# Next we establish a list of backends that we want to run on.  These must be strings.
backends = ['ibmq_valencia', 'ibmq_santiago', 'ibmq_vigo', 'ibmqx2', 'ibmq_ourense', 'ibmq_athens']
#Running the results and writing the info on a csv file
A = CSVWriter(qc, backends, '000', 'H', 2400, 'testSTD.csv')
A.generate_csv()

Initializing Jobs...
Running qc on ibmq_valencia for 2400 shots...
done.
Running qc on ibmq_santiago for 2400 shots...
done.
Running qc on ibmq_vigo for 2400 shots...
done.
Running qc on ibmqx2 for 2400 shots...
done.
Running qc on ibmq_ourense for 2400 shots...
done.
Running qc on ibmq_athens for 2400 shots...
done.


In [3]:
#Run 1 Dec 13, 2020
#sh_va={'*0*': 1982, '*1*': 418} 
#sh_sa={'*0*': 1945, '*1*': 455} 
#sh_vi={'*0*': 2040, '*1*': 360} 
#sh_yt={'*0*': 1869, '*1*': 531} 
#sh_ou={'*0*': 1977, '*1*': 423} 

#Run 2 Dec 15, 2020
#sh_va={'0': 1980, '1': 420} 
#sh_sa={'0': 1049, '1': 1351} 
#sh_vi={'0': 1987, '1': 413} 
#sh_yt={'0': 1895, '1': 505} 
#sh_ou={'0': 1993, '1': 407} 
#sh_at={'0': 2091, '1': 309} 


In [2]:
#Run 3 Dec 29, 2020
sh_va={'*0*': 1924, '*1*': 476}
sh_sa={'*0*': 2080, '*1*': 320}
sh_vi={'*0*': 2118, '*1*': 282}
sh_yt={'*0*': 1624, '*1*': 776}
sh_ou={'*0*': 1977, '*1*': 423}
sh_at={'*0*': 2008, '*1*': 392}


In [3]:
colors= colors= ['blue','orange','green', 'red','purple', 'cyan']
shhist=plot_histogram([sh_va,sh_sa,sh_vi,sh_yt,sh_ou, sh_at],color=colors, title='Standard gates decomposition with W=H',figsize=(10,10))
yax = shhist.axes[0]
yax.set_ylim(0,1)
ax = shhist.gca()

Legend_Lines = [Line2D([0], [0], color=colors[0], lw=4),
                Line2D([0], [0], color=colors[1], lw=4),
                Line2D([0], [0], color=colors[2], lw=4),
                Line2D([0], [0], color=colors[3], lw=4),
                Line2D([0], [0], color=colors[4], lw=4),
                Line2D([0], [0], color=colors[5], lw=4)]

# Stick them in legend
shhist.legend(Legend_Lines, ['ibmq_valencia','ibmq_santiago','ibmq_vigo','ibmqx2','ibmq_ourense', 'ibmq_athens'], bbox_to_anchor=(0.95, 0.95), bbox_transform=ax.transAxes)

#Profit
shhist.savefig('3SH.png')
